In [1]:
### imports
import json
import tensorflow as tf
from sklearn.model_selection import train_test_split
import asl

# %load_ext autoreload
# %autoreload 2

### pretty print = off
# %pprint

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## Metadata

In [ ]:
Meta = asl.metadata('train')
MetaS = asl.metadata('supplemental')

## Selecting datafiles

In [ ]:
file = str(MetaS.files[1])
Datafile = asl.datafile(MetaS, file)

## Selecting videos

In [ ]:
vid = Datafile.videos[0]
W = asl.video(Datafile, vid)
asl.hand_video(W)

# Label and Pickle 

In [ ]:
#Label(V,M)

# change label
def changelabel(self, i, letter):
    self.data_labels[i] = letter
    
# pickle function
def pickle(D,V):
    seq_id = str(V.id)
    Frames = V.frames
    D[seq_id] = V.data_labels
    
#for i in range(81,82):
    #changelabel(V,i,'?')

In [ ]:
#for i in range(81,82):
    #changelabel(V,i,'?')
# pickle(D,V)

# Model

### Prepare data

In [ ]:
# initialize
X, y = [], []

# loop over file ids
def generate_training_data(file):
    
    # file as string
    sfile = str(file)
    
    # load dictionary
    dstring = 'asl/Labels/Labels' + sfile + '.json'
    with open( dstring, 'r') as f:
        train = json.load(f)
    
    # select parameters 
    Met = MetaS if file in MetaS.files else Meta
    Df = asl.datafile(Met, sfile )
    videos = train[sfile]
    
    # videos 
    for vid in videos:
        # load video
        V = asl.video(Df, int(vid))
        N = asl.normal_hand(V)
        pN = videos[vid]
        
        # loop
        for i in range(len(N)):
            if pN[i] != '?':
                X.append( N[i].numpy() )
                y.append( asl.Encode( pN[i] ) )

In [ ]:
#training_files = [105143404, 128822441, 33432165, 86446671]
training_files = [ 86446671, 33432165 ]
import time 
start_time = time.time()
# training_files = [ 86446671 ]
for file in training_files:
    generate_training_data(file)
print("--- %s seconds ---" % (time.time() - start_time))   

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) 

### Model 

In [ ]:
# pause

import numpy as np
import tensorflow as tf
X = np.array(X)
y = np.array(y)

In [ ]:
# Model
k = 59
M = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(20, 3)),
    tf.keras.layers.Dense(6*k, activation='relu'),
    tf.keras.layers.Dense(k)])

# Compile Model
M.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
# fit model
M.fit( X, y, epochs = 400, verbose=0 )
# M.fit( X_train, y_train, epochs = 500, verbose=0 )

In [ ]:
Loss, Acctrain = M.evaluate( X, y, verbose=2)
print('accuracy:', Acctrain)

In [ ]:
#Loss, Acctrain = M.evaluate(X_train, y_train, verbose=2)
#Loss, Acctest = M.evaluate(X_test, y_test, verbose=2)
#print('\nTrain accuracy:', Acctrain)
#print('\nTest accuracy:', Acctest)

 ### save + load past models 

In [ ]:
# save model 
# M.save("alphabetmodelmain.h5")

# load model 
# M = tf.keras.models.load_model('asl/alphabetmodel1.h5')

In [ ]:
path = 'Examples/IMG_0970.MOV'
V = asl.video_file(path)